# Long/short portfolio construction model trading strategy example

This is an example notebook how to construct a momentum based portfolio construction strategy
using [Trading Strategy framework](https://tradingstrategy.ai/docs/) and backtest it for DeFi tokens.

This backtest uses alpha model approach where each trading pair has a signal
and based on the signal strenghts we construct new portfolio weightings
for the upcoming week.


## Strategy parameter set up

Set up the parameters used in in this strategy backtest study.

- Backtested blockchain, exchange and trading pair
- Backtesting period
- Strategy parameters for EMA crossovers

In [1]:
import datetime
import pandas as pd

from tradingstrategy.chain import ChainId
from tradingstrategy.timebucket import TimeBucket
from tradeexecutor.strategy.cycle import CycleDuration
from tradeexecutor.strategy.strategy_module import StrategyType, TradeRouting, ReserveCurrency

# Tell what trade execution engine version this strategy needs to use
trading_strategy_engine_version = "0.3"

# What kind of strategy we are running.
# This tells we are going to use
trading_strategy_type = StrategyType.managed_positions

# How our trades are routed.
# PancakeSwap basic routing supports two way trades with BUSD
# and three way trades with BUSD-BNB hop.
trade_routing = TradeRouting.ignore

# Set cycle to 7 days and look back the momentum of the previous candle
trading_strategy_cycle = CycleDuration.cycle_7d
momentum_lookback_period = datetime.timedelta(days=7)

# Hold top 3 coins for every cycle
max_assets_in_portfolio = 4

# Leave 20% cash buffer
value_allocated_to_positions = 0.80

# Set 33% stop loss over mid price
stop_loss = None

# Set 5% take profit over mid price
take_profit = None

# The weekly price must be up 2.5% for us to take a long position
minimum_mometum_threshold = 0.025

# Don't bother with trades that would move position
# less than 300 USD
minimum_rebalance_trade_threshold = 300

# Use hourly candles to trigger the stop loss
stop_loss_data_granularity = TimeBucket.h1

# Strategy keeps its cash in USDC
reserve_currency = ReserveCurrency.usdc

# The duration of the backtesting period
backtest_start_at = datetime.datetime(2020, 11, 1)
backtest_end_at = datetime.datetime(2023, 1, 31)

# Start with 10,000 USD
initial_deposit = 10_000


## Strategy logic and trade decisions

`decide_trades` function decide what trades to take.
In this example, we calculate two exponential moving averages
(EMAs) and make decisions based on those.


In [2]:
from tradeexecutor.strategy.trading_strategy_universe import translate_trading_pair, TradingStrategyUniverse
from typing import List, Dict, Counter

from tradingstrategy.universe import Universe
from tradeexecutor.strategy.weighting import weight_by_1_slash_n
from tradeexecutor.strategy.alpha_model import AlphaModel
from tradeexecutor.state.trade import TradeExecution
from tradeexecutor.strategy.pricing_model import PricingModel
from tradeexecutor.strategy.pandas_trader.position_manager import PositionManager
from tradeexecutor.state.state import State


def decide_trades(
        timestamp: pd.Timestamp,
        universe: TradingStrategyUniverse,
        state: State,
        pricing_model: PricingModel,
        cycle_debug_data: Dict) -> List[TradeExecution]:

    # Create a position manager helper class that allows us easily to create
    # opening/closing trades for different positions
    position_manager = PositionManager(timestamp, universe, state, pricing_model)

    alpha_model = AlphaModel(timestamp)

    # Watch out for the inclusive range and include and avoid peeking in the future
    adjusted_timestamp = timestamp - pd.Timedelta(seconds=1)
    start = adjusted_timestamp - momentum_lookback_period - datetime.timedelta(seconds=1)
    end = adjusted_timestamp 

    candle_universe = universe.candles
    pair_universe = universe.pairs

    # Get candle data for all candles, inclusive time range
    candle_data = candle_universe.iterate_samples_by_pair_range(start, end)

    # Iterate over all candles for all pairs in this timestamp (ts)
    for pair_id, pair_df in candle_data:

        # We should have candles for range start - end,
        # where end is the current strategy cycle timestamp 
        # and start is one week before end.
        # Because of sparse data we may have 0, 1 or 2 candles
        first_candle = pair_df.iloc[0]
        last_candle = pair_df.iloc[-1]

        # How many candles we are going to evaluate
        candle_count = len(pair_df)

        assert last_candle["timestamp"] < timestamp, "Something wrong with the data - we should not be able to peek the candle of the current timestamp, but always use the previous candle"

        open = last_candle["open"]
        close = last_candle["close"]

        # DEXPair instance contains more data than internal TradingPairIdentifier
        # we use to store this pair across the strategy
        dex_pair = pair_universe.get_pair_by_id(pair_id)
        pair = translate_trading_pair(dex_pair)

        # We define momentum as how many % the trading pair price gained during
        # the momentum window
        momentum = (close - open) / open
        
        # This pair has not positive momentum,
        # we only buy when stuff goes up
        if momentum <= minimum_mometum_threshold:
            continue

        if momentum < 0:
            if not universe.can_open_short(
                timestamp,
                pair
            ):
                # Lending markets are not yet available at this point of time
                # for this trading pair
                continue

        alpha_model.set_signal(
            pair,
            momentum,
            stop_loss=stop_loss,
            take_profit=take_profit,
        )

    # Select max_assets_in_portfolio assets in which we are going to invest
    # Calculate a weight for ecah asset in the portfolio using 1/N method based on the raw signal
    alpha_model.select_top_signals(max_assets_in_portfolio)
    alpha_model.assign_weights(method=weight_by_1_slash_n)
    alpha_model.normalise_weights()

    # Load in old weight for each trading pair signal,
    # so we can calculate the adjustment trade size
    alpha_model.update_old_weights(state.portfolio)

    # Calculate how much dollar value we want each individual position to be on this strategy cycle,
    # based on our total available equity
    portfolio = position_manager.get_current_portfolio()
    portfolio_target_value = portfolio.get_total_equity() * value_allocated_to_positions
    alpha_model.calculate_target_positions(position_manager, portfolio_target_value)

    # Shift portfolio from current positions to target positions
    # determined by the alpha signals (momentum)
    trades = alpha_model.generate_rebalance_trades_and_triggers(
        position_manager,
        min_trade_threshold=minimum_rebalance_trade_threshold,  # Don't bother with trades under 300 USD
    )

    # Record alpha model state so we can later visualise our alpha model thinking better
    state.visualisation.add_calculations(timestamp, alpha_model.to_dict())

    return trades


## Set up the market data client

The [Trading Strategy market data client](https://tradingstrategy.ai/docs/programming/api/client/index.html)
is the Python library responsible for managing the data feeds needed to run the backtest.None

We set up the market data client with an API key.

[If you do not have an API key yet, you can register one](https://tradingstrategy.ai/trading-view/backtesting).

In [3]:
from tradingstrategy.client import Client

client = Client.create_jupyter_client()

Started Trading Strategy in Jupyter notebook environment, configuration is stored in /Users/moo/.tradingstrategy


## Setup trading universe

- Load all lending reserves for our chain
- Load matching trading pairs we can use to do our long/short trades

In [4]:
from tradingstrategy.client import Client

from tradeexecutor.strategy.trading_strategy_universe import TradingStrategyUniverse, load_trading_and_lending_data
from tradeexecutor.strategy.execution_context import ExecutionContext
from tradeexecutor.strategy.execution_context import ExecutionMode
from tradeexecutor.strategy.universe_model import UniverseOptions

def create_trading_universe(
        ts: datetime.datetime,
        client: Client,
        execution_context: ExecutionContext,
        universe_options: UniverseOptions,
) -> TradingStrategyUniverse:

    # for all lending markets on a relevant time period
    dataset = load_trading_and_lending_data(
        client,
        execution_context=ExecutionContext(mode=ExecutionMode.data_research),
        universe_options=universe_options,
        # Ask for all Polygon data
        chain_id=ChainId.polygon,
        # We limit ourselves to price feeds on Uniswap v3 and Quickswap on Polygon,
        # as there are multiple small or dead DEXes on Polygon
        # which also have price feeds but not interesting liquidity
        exchange_slugs={"uniswap-v3", "quickswap"},
        reserve_asset_symbols={"USDC"},
        asset_symbols={"LINK", "WMATIC", "WETH"},
        volatile_only=True,
    )

    # Filter down the dataset to the pairs we specified
    universe = TradingStrategyUniverse.create_from_dataset(dataset)

    return universe

universe = create_trading_universe(
    datetime.datetime.utcnow(),
    client,
    ExecutionContext(mode=ExecutionMode.backtesting),
    UniverseOptions(start_at=backtest_start_at, end_at=backtest_end_at),
)


<LendingReserve #2 for asset LINK in protocol aave_v3 on Polygon >
<LendingReserve #3 for asset USDC in protocol aave_v3 on Polygon >
<LendingReserve #5 for asset WETH in protocol aave_v3 on Polygon >
<LendingReserve #8 for asset WMATIC in protocol aave_v3 on Polygon >


## Available trading universe

Inspect trading pairs and lending markets we are going to backtest.


In [5]:
from IPython.display import HTML

from tradeexecutor.analysis.universe import analyse_long_short_universe
from tradingstrategy.utils.jupyter import format_links_for_html_output

df = analyse_long_short_universe(universe)
df = format_links_for_html_output(df, ("Price data page", "Lending rate page",))

display(HTML(df.to_html(escape=False)))

,Lending asset,Stablecoin,Best trading pair,Lending available at,Trading available at,Price data page,Lending rate page
0,LINK,no,LINK-USDC at 5 BPS fee tier on Uniswap v3,2022-03-16,2022-08-21,View,View
1,USDC,yes,No AMM pools found,2022-03-16,-,View,View
2,WETH,no,WETH-USDC at 5 BPS fee tier on Uniswap v3,2022-03-16,2021-12-21,View,View
3,WMATIC,no,WMATIC-USDC at 5 BPS fee tier on Uniswap v3,2022-03-12,2022-07-07,View,View


## Run backtest

Run backtest using giving trading universe and strategy function.

- Running the backtest outputs `state` object that contains all the information
on the backtesting position and trades.
- The trade execution engine will download the necessary datasets to run the backtest.
  The datasets may be large, several gigabytes.


In [6]:
import logging

from tradeexecutor.backtest.backtest_runner import run_backtest_inline

state, universe, debug_dump = run_backtest_inline(
    name="Portfolio construction model example strategy",
    start_at=backtest_start_at,
    end_at=backtest_end_at,
    client=client,
    cycle_duration=trading_strategy_cycle,
    decide_trades=decide_trades,
    create_trading_universe=create_trading_universe,
    initial_deposit=initial_deposit,
    reserve_currency=reserve_currency,
    trade_routing=trade_routing,
    log_level=logging.WARNING,
    universe=universe,
)

trade_count = len(list(state.portfolio.get_all_trades()))
print(f"Backtesting completed, backtested strategy made {trade_count} trades")

  0%|          | 0/70934400 [00:00<?, ?it/s]

CandleSampleUnavailable: Could not fetch price for <Pair LINK-USDC spot_market_hold at 0x70cee55c98f6da2685807611f115ea737d4a248e (0.3000% fee) on exchange 0x5757371414417b8c6caad45baef941abc7d3ab32> at 2021-03-15 00:00:00

This is usually due to sparse candle data - trades have not been made or the blockchain was halted during the price look-up period.
Because there are no trades we cannot determine what was the correct asset price using 2 days 00:00:00 data tolerance delay.

You can work around this by checking that any trading pair candles are fresh enough in your decide_trades() function
or increase the parameter in BacktestSimplePricingModel(data_delay_tolerance) or run_backtest_inline(data_delay_tolerance)


## Examine backtest results

Examine `state` that contains 
- All actions the trade executor took
- Visualisation and diagnostics data associated with the actity

We plot out a chart that shows
- The price action
- When the strategy made buys or sells

In [ ]:
print(f"Positions taken: {len(list(state.portfolio.get_all_positions()))}")
print(f"Trades made: {len(list(state.portfolio.get_all_trades()))}")

## Benchmarking the strategy performance

Here we benchmark the strategy performance against some baseline scenarios.

- Buy and hold ETH
- Buy and hold US Dollar (do nothing)


In [ ]:
from tradeexecutor.visual.benchmark import visualise_benchmark

eth_usd = universe.get_pair_by_human_description((ChainId.ethereum, "uniswap-v2", "WETH", "USDC"))
eth_candles = universe.universe.candles.get_candles_by_pair(eth_usd.internal_id)
eth_price = eth_candles["close"]

fig = visualise_benchmark(
    state.name,
    portfolio_statistics=state.stats.portfolio,
    all_cash=state.portfolio.get_initial_deposit(),
    buy_and_hold_asset_name="ETH",
    buy_and_hold_price_series=eth_price,
    start_at=backend_start_at,
    end_at=backend_end_at
)

fig.show()

## Analysing  the strategy success

Here we calculate statistics on how well the strategy performed.

- Won/lost trades
- Timeline of taken positions with color coding of trade performance


In [ ]:
from tradeexecutor.analysis.trade_analyser import build_trade_analysis

analysis = build_trade_analysis(state.portfolio)

### Strategy summary

Overview of strategy performance



In [ ]:
from IPython.core.display_functions import display

summary = analysis.calculate_summary_statistics()

with pd.option_context("display.max_row", None):
    display(summary.to_dataframe())

### Alpha model timeline analysis

We display a table that shows how alpha model performermed over time.

- Events are in the timeline order

- Display portfolio weightings at each strategy cycle

- Show performance of individidual positions taken during this cycle

- Stop loss events are shown as their own rows

The visualisation will have the a column for each asset being traded. The cell of a column contains the following elements

- Asset **price change** (up/down) since the last strategy cycle

- **Price** of the asset at the start of the strategy cycle

- **Weight** % as the part of the portfolio that was chosen for the next cycle

- **Open** value of the position as USD

- **Adjust** value of the position as USD -  to get to the target weight - how much asset needs to be bought or sold

- **Close** the remaining value of the position as USD 

- **Profit**: The life time profit of the position so far

Position opening and closing rules

- A position may contain multiple trades

- Unlike on some exchanges, we adjust existing positions by increasing or decreasing them - a trade can count against existing position

- Posions are opened if there are no existing holdings of an asset

- The position is adjusted over the time

- The position is closed when the remaining assets of the position are sold

In [ ]:
from tradeexecutor.analysis.alpha_model_analyser import render_alpha_model_plotly_table, create_alpha_model_timeline_all_assets

timeline = create_alpha_model_timeline_all_assets(state, universe, new_line="<br>")
figure, table = render_alpha_model_plotly_table(timeline)

figure.update_layout(
    autosize=True,
    height=800,)

display(figure)

## Single pair trade, entry and exit analyses

Plot a single pair from our trading universe and show our entries and exits for this particular pair.

- Analyse ETH/USDC entries and exits
- We use 1h candles, as weekly candles used for strategy decisions do not have enough time resolution - we downloaded 1h candles earlier to be used as a trigger signal for take profit/stop loss

In [ ]:
from tradeexecutor.visual.single_pair import visualise_single_pair
from tradingstrategy.charting.candle_chart import VolumeBarMode

eth_usd = universe.get_pair_by_human_description((ChainId.ethereum, "uniswap-v2", "WETH", "USDC"))
eth_usd_candles = universe.backtest_stop_loss_candles.get_candles_by_pair(eth_usd.internal_id)

fig = visualise_single_pair(
    state,
    pair_id=eth_usd.internal_id,
    title="ETH/USDC individual trades",
    candle_universe=eth_usd_candles,
    start_at=start_at,
    end_at=end_at,
    volume_bar_mode=VolumeBarMode.hidden,
)

fig.show()

In [ ]:
from tradeexecutor.visual.single_pair import visualise_single_pair_positions_with_duration_and_slippage

fig = visualise_single_pair_positions_with_duration_and_slippage(
    state,
    pair_id=eth_usd.internal_id,
    title="ETH/USDC entries and exits",
    candles=eth_usd_candles,
    start_at=start_at,
    end_at=end_at,
)

fig.show()

## Finishing notes

Print out a line to signal the notebook finished the execution successfully.

In [ ]:
print("All ok")